In [ ]:
import numpy as np
from astropy.table import Table

We use the constants for different IA models as inputs

In [ ]:
A_IA = 0.0
b_TA = 0.0
C_2= -1.0


The intensity of the NLA, delta-NLA and TT models can be chosen by tweaking the numbers above, this can probably be changed for some calculation involving weights given to each model but that's up for discussion, for now, we setup for the observed epsilons with and without noise, using the provided parameters

In [ ]:
for i in range(1,6):
#for i in np.arange(5)+1:

    #V0
    #catalog = Table.read('../../GalCat/StageIV_nz/V0/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i),
    #catalog = Table.read('../../GalCat/StageIV_nz/V0/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_smooth.dat'.format(i),
    catalog = Table.read('../../GalCat/SRD-Y1/random_pos/GalCat_tomo{}_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.1Mpc.asc'.format(i),

    #V1
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc'.format(i),
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'.format(i),
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc'.format(i),
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'.format(i),
    #catalog = Table.read('/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_smooth.dat'.format(i),
                     format='ascii')  # descriptions=['ra_deg', 'dec_deg', 'shear1', 'shear2', 'weight', 'z_true', 'delta', 's11', 's22', 's12'])

    #V0
    #epsilons = Table.read('../../GalCat/StageIV_nz/V0/epsilon_{}_0.06GpAM.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V0/epsilon_{}_0.06GpAM_V2.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V0/epsilon_{}_0.06GpAM_smooth_V2.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    epsilons = Table.read('../../GalCat/SRD-Y1/random_pos/epsilon_{}_0.6GpAM_smooth0.1Mpc.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']

    #V1
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.6GpAM_linear_bias_smooth.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth_1Mpc.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']

    
    ra=np.asarray(catalog['col1'])
    dec=np.asarray(catalog['col2'])
    weight=np.asarray(catalog['col5'])
    z=np.asarray(catalog['col6'])
    gamma1 = np.asarray(catalog['col3'])
    gamma2 = np.asarray(catalog['col4'])
    e1_NLA = np.asarray(epsilons['e1_NLA'])
    e2_NLA = np.asarray(epsilons['e2_NLA'])
    e1_TT = np.asarray(epsilons['e1_TT'])
    e2_TT = np.asarray(epsilons['e2_TT'])
    e1_noise = np.asarray(epsilons['e1_noise'])
    e2_noise = np.asarray(epsilons['e2_noise'])
    delta = np.asarray(catalog['col7'])
    epsilon_NLA = np.empty(e1_NLA.shape, dtype = complex)
    epsilon_NLA.real = A_IA * e1_NLA  
    epsilon_NLA.imag = A_IA * e2_NLA  
    epsilon_TT = np.empty(e1_TT.shape, dtype = complex)
    epsilon_TT.real = C_2 * e1_TT  
    epsilon_TT.imag = C_2 * e2_TT
    epsilon_random = np.empty(e1_TT.shape, dtype = complex)
    epsilon_random.real = e1_noise
    epsilon_random.imag = e2_noise
    g = np.empty(gamma1.shape, dtype=complex)
    g.real = gamma1
    g.imag = gamma2
    
    #calibrate delta:
    #delta /=2.5
    #delta*=0.0
    
    #calibrate the low-redshift e_TT:
    epsilon_TT[z<0.5]/=2.5

    #V1 mocks further require an overall calibration (not sure why)
    #epsilon_TT /=20.0

    
    #And finally, we calculate the observed epsilons using the summation as in equation 11 of the paper
    # The V1 mocks already have the delta-weighted term, so we keep it or we undo it. 
    # In the later case, we must use the same b_TA parameter as that used in the construction of the mocks
    # Keep delta-weights:
    epsilon_IA_TATT = epsilon_NLA + epsilon_TT
    # Undo delta-weights:
    #epsilon_IA_TATT = epsilon_NLA / (1 + b_TA * delta) + epsilon_TT
    #epsilon_IA_TATT /= (1 + b_TA * delta)
    epsilon_int = (epsilon_IA_TATT + epsilon_random)/(1 + np.conjugate(epsilon_IA_TATT) * epsilon_random)
    epsilon_obs = (epsilon_int + g)/(1 + epsilon_int * np.conjugate(g))
    epsilon_obs_no_noise = (epsilon_IA_TATT + g)/(1 + epsilon_IA_TATT * np.conjugate(g))
   
    fin_epsilon = Table([ra,dec,z,gamma1, gamma2, weight,delta,epsilon_obs.real,epsilon_obs.imag,epsilon_obs_no_noise.real,epsilon_obs_no_noise.imag,epsilon_IA_TATT.real,epsilon_IA_TATT.imag], names = ('ra','dec','z','g1', 'g2', 'weight','delta','e_obs_1','e_obs_2', 'e_obs_no_noise_1','e_obs_no_noise_2','e_IA_TATT_1', 'e_IA_TATT_2' ))
    #fin_epsilon.write('/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i), format='ascii', overwrite=True)    

    #V0:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM.dat'.format(i), format='ascii', overwrite=True)    
    fin_epsilon.write('../../GalCat/SRD-Y1/random_pos/fin_epsilon_IA_{}_0.6GpAM_smooth0.1Mpc.dat'.format(i), format='ascii', overwrite=True)    

    #V0-TT                     
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM_TT.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM_TT_V2.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM_TT_V2corr.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM_TT_smooth_V2corr.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM_TT_smooth_V3corr.dat'.format(i), format='ascii', overwrite=True)    

    #V1:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.06GpAM_linear_bias.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.06GpAM_linear_bias_smooth.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.06GpAM_linear_bias_smooth_1Mpc.dat'.format(i), format='ascii', overwrite=True)    

    #V1-NLA:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth_NLA.dat'.format(i), format='ascii', overwrite=True)    
    
    print('Done tomo',i)

In [ ]:
for i in range(1,6):
#for i in np.arange(5)+1:

    #V0
    #catalog = np.loadtxt('../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.1Mpc.asc')
    
    #V1
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc'.format(i),
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'.format(i),
    #catalog = Table.read('../../GalCat/StageIV_nz/V1/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc'.format(i),
    #catalog = np.loadtxt('../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc')
    #catalog = np.loadtxt('../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc')
    catalog = np.loadtxt('../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta1_smooth0.1Mpc.asc')
    #catalog = np.loadtxt('../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(i)+'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta2_smooth0.1Mpc.asc')

    #V0
    #epsilons = Table.read('../../GalCat/StageIV_nz/V0/epsilon_{}_0.06GpAM.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/SRD-Y1/random_pos/epsilon_{}_0.6GpAM_smooth0.1Mpc.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']

    #V1
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.6GpAM_linear_bias_smooth.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.6GpAM_linear_bias_smooth_bta2.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = np.loadtxt('../../GalCat/StageIV_nz/V1/epsilon_'+str(i)+'_0.6GpAM_linear_bias_smooth.asc')  #descriptions=['e1','e2','e1_noise','e2_noise']
    #epsilons = Table.read('../../GalCat/StageIV_nz/V1/epsilon_{}_0.06GpAM_linear_bias_smooth_1Mpc.asc'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']    
    epsilons = Table.read('../../GalCat/SRD-Y1/lin_bias_pos/epsilon_{}_0.6GpAM_smooth0.1Mpc.asc'.format(i), format='ascii')
    #epsilons = Table.read('../../GalCat/SRD-Y1/lin_bias_pos/epsilon_{}_0.6GpAM_smooth_bta2_0.1Mpc.asc'.format(i), format='ascii')
    
    ra=np.asarray(catalog[:,0])
    dec=np.asarray(catalog[:,1])
    weight=np.asarray(catalog[:,4])
    z=np.asarray(catalog[:,5])
    gamma1 = np.asarray(catalog[:,2])
    gamma2 = np.asarray(catalog[:,3])
    e1_NLA = np.asarray(epsilons['e1_NLA'])
    e2_NLA = np.asarray(epsilons['e2_NLA'])
    e1_TT = np.asarray(epsilons['e1_TT'])
    e2_TT = np.asarray(epsilons['e2_TT'])
    e1_noise = np.asarray(epsilons['e1_noise'])
    e2_noise = np.asarray(epsilons['e2_noise'])
    delta = np.asarray(catalog[:,6])
    epsilon_NLA = np.empty(e1_NLA.shape, dtype = complex)
    epsilon_NLA.real = A_IA * e1_NLA  
    epsilon_NLA.imag = A_IA * e2_NLA  
    epsilon_TT = np.empty(e1_TT.shape, dtype = complex)
    epsilon_TT.real = C_2 * e1_TT  
    epsilon_TT.imag = C_2 * e2_TT
    epsilon_random = np.empty(e1_TT.shape, dtype = complex)
    epsilon_random.real = e1_noise
    epsilon_random.imag = e2_noise
    g = np.empty(gamma1.shape, dtype=complex)
    g.real = gamma1
    g.imag = gamma2
    
    #------------------
    #calibrate delta:
    #delta*=0.0
    
    #calibrate e_TT:
    epsilon_TT[z<0.5]/=2.5
    epsilon_TT/=20
    #------------------
    
    #And finally, we calculate the observed epsilons using the summation as in equation 11 of the paper
    # The V1 mocks already sample the galaxy by linearly tracing the matter field, so they already have the 
    # delta-weighted term. 
    epsilon_IA_TATT = epsilon_NLA + epsilon_TT
    #Undo delta-weight:
    #epsilon_IA_TATT = epsilon_NLA / (1 + b_TA * delta) + epsilon_TT
    #epsilon_IA_TATT /= (1 + b_TA * delta) 
    epsilon_int = (epsilon_IA_TATT + epsilon_random)/(1 + np.conjugate(epsilon_IA_TATT) * epsilon_random)
    epsilon_obs = (epsilon_int + g)/(1 + epsilon_int * np.conjugate(g))
    epsilon_obs_no_noise = (epsilon_IA_TATT + g)/(1 + epsilon_IA_TATT * np.conjugate(g))
   
    fin_epsilon = Table([ra,dec,z,gamma1, gamma2, weight,delta,epsilon_obs.real,epsilon_obs.imag,epsilon_obs_no_noise.real,epsilon_obs_no_noise.imag,epsilon_IA_TATT.real,epsilon_IA_TATT.imag], names = ('ra','dec','z','g1', 'g2', 'weight','delta','e_obs_1','e_obs_2', 'e_obs_no_noise_1','e_obs_no_noise_2','e_IA_TATT_1', 'e_IA_TATT_2' ))
    
    #fin_epsilon.write('/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i), format='ascii', overwrite=True)    

    #V0:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V0/fin_epsilon_IA_{}_0.06GpAM.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/random_pos/fin_epsilon_IA_{}_0.6GpAM_smooth0.1Mpc.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/random_pos/fin_epsilon_IA_{}_0.6GpAM_smooth0.1Mpc_m1.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/random_pos/fin_epsilon_IA_{}_0.6GpAM_smooth0.1Mpc_TT.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/random_pos/fin_epsilon_IA_{}_0.6GpAM_smooth0.1Mpc_TT_m1.dat'.format(i), format='ascii', overwrite=True)    

    
    #V1-deltaNLA:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.06GpAM_linear_bias.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.06GpAM_linear_bias_smooth.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.06GpAM_linear_bias_smooth_1Mpc.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.5Mpc.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.5Mpc_m1.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.5Mpc_bta2.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.1Mpc_bta2_m1.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.5Mpc_m1.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.5Mpc_TT.dat'.format(i), format='ascii', overwrite=True)    
    fin_epsilon.write('../../GalCat/SRD-Y1/lin_bias_pos/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth0.1Mpc_TT_m1.dat'.format(i), format='ascii', overwrite=True)    

    #V1-NLA:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth_NLA.dat'.format(i), format='ascii', overwrite=True)    

    #V1-TT:
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth_TT.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth_TT_V3.dat'.format(i), format='ascii', overwrite=True)    
    #fin_epsilon.write('../../GalCat/StageIV_nz/V1/fin_epsilon_IA_{}_0.6GpAM_linear_bias_smooth_TT_nodelta.dat'.format(i), format='ascii', overwrite=True)    
    
    
    print('Done tomo',i)